In [ ]:
import cv2
import numpy as np

# Load the video
video = cv2.VideoCapture(r"C:\Users\HP\Downloads\istockphoto-1764948156-640_adpp_is.mp4")

# Read the first frame
ret, prev_frame = video.read()
if not ret:
    print("Error: Unable to read the video")
    exit()

# Convert to grayscale
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
)

# Detect good features and create a mask image
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask=None, maxCorners=100, qualityLevel=0.3, minDistance=5, blockSize=7)
mask = np.zeros_like(prev_frame)



# Define video writer parameters
output_path = r"C:\Users\HP\Downloads\output_optical_flow.mp4"  # Path to save the output video
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))  # Frames per second from the input video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Process the video frame by frame
while True:
    ret, curr_frame = video.read()
    if not ret:
        break

    # Convert current frame to grayscale
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow using Lucas-Kanade method
    curr_corners, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_corners, None, **lk_params)

    # Filter valid points
    good_old = prev_corners[status == 1]
    good_new = curr_corners[status == 1]

    # Draw motion vectors
    for old, new in zip(good_old, good_new):
        x_old, y_old = old.ravel()
        x_new, y_new = new.ravel()
        mask = cv2.line(mask, (int(x_old), int(y_old)), (int(x_new), int(y_new)), (0, 255, 0), 2)
        curr_frame = cv2.circle(curr_frame, (int(x_new), int(y_new)), 5, (0, 0, 255), -1)

    # Combine the frame and mask for visualization
    output = cv2.add(curr_frame, mask)

    # Write the processed frame to the output video
    out.write(output)

    # Display the result
    cv2.imshow('L K O F', output)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    # Update previous frame and corners for the next iteration
    prev_gray = curr_gray.copy()
    prev_corners = good_new.reshape(-1, 1, 2)

# Release resources
video.release()
out.release()  # Release the VideoWriter object
cv2.destroyAllWindows()

print(f"Video saved successfully at: {output_path}")

Video saved successfully at: C:\Users\HP\Downloads\output_optical_flow.mp4
